In [1]:
import json
from bs4 import BeautifulSoup

import scrapy
import scrapy.crawler as crawler
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.getLogger('scrapy').propagate = False

SCRAPY_RESULT_FILE = "graph.json"
SCRAPY_PAGES_COUNT = 10100
SCRAPY_LINKS_LIMIT = 100

## 1. Скачивание

Скачаем примерно 10 тысяч страниц английской википедии. Стоит отметить, что класс `CrawlSpider` определяет метод `parse`, в котором используется `set` для исключения переходов по одним и тем же ссылкам несколько раз. 

In [2]:
class CustomSpider(CrawlSpider):
    name = "custom_spider"
    
    allowed_domains = ["en.wikipedia.org"]
    start_urls = [
        "https://en.wikipedia.org/wiki/Information_retrieval",
        "https://en.wikipedia.org/wiki/Mein_Kampf",
        "https://en.wikipedia.org/wiki/Soviet_Union",
        "https://en.wikipedia.org/wiki/Nineteen_Eighty-Four",
        "https://en.wikipedia.org/wiki/The_Hero_with_a_Thousand_Faces",
    ]
    rules = (
        Rule(LinkExtractor(allow="https://en\.wikipedia\.org/wiki/" + \
                                 "(?!(File|Talk|Category|Portal|Special|Wikipedia|Help|Draft|Main_Page)).+",
                           restrict_xpaths='//div[@id="mw-content-text"]',
                           canonicalize=True,
                           unique=True),
             process_links=lambda links: links[:SCRAPY_LINKS_LIMIT],
             callback="parse_item", 
             follow=True),
    )
    
    custom_settings = {
        "CLOSESPIDER_PAGECOUNT": SCRAPY_PAGES_COUNT,
        "CLOSESPIDER_ERRORCOUNT": 0,
        "CONCURRENT_REQUESTS": 16
    }

    def parse_item(self, response):
        try:
            title = response.xpath('//h1[@id="firstHeading"]/text()').extract_first()
            snippet = BeautifulSoup(response.xpath('//p[1]').extract_first(), "lxml").text[:255] + "..."
            links = [lnk.url for rule in self._rules 
                     for lnk in rule.process_links(rule.link_extractor.extract_links(response))]
            return {'url': response.url, 'title': title, 'snippet': snippet, 'links': links}
        except:
            return None

In [3]:
runner = crawler.CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': SCRAPY_RESULT_FILE
})
runner.crawl(CustomSpider)
runner.start()

## 2. Построение графа

При парсинге мы сохраняли для текущей страницы $100$ ссылок на ней, но не по всем из них нам удастся перейти. В граф будем добавлять только те урлы, по которым перешёл наш паук. В получившемся ориентированном графе $9969$ страниц и $43976$ рёбер.

In [4]:
graph_json = json.load(open(SCRAPY_RESULT_FILE), encoding='utf-8')
nodes = {x["url"] for x in graph_json}

G = nx.DiGraph()
G.add_nodes_from(nodes)

def get_edges():
    for line in graph_json:
        source, targets = line["url"], line["links"]

        for target in targets:
            if target in nodes:
                yield source, target

G.add_edges_from(get_edges())

In [5]:
print("Number of nodes: %d" % G.number_of_nodes())
print("Number of edges: %d" % G.number_of_edges())
print("Max out degree: %d" % max(deg for _, deg in G.out_degree(nodes)))
print("Max in degree: %d" % max(deg for _, deg in G.in_degree(nodes)))

Number of nodes: 9969
Number of edges: 43976
Max out degree: 23
Max in degree: 135


## 3. PageRank

__PageRank__ - характеризует "важность" страницы. Чем больше ссылок на страницу, тем она "важнее". При этом "вес" документа определяется через веса ссылок на неё, и можно регулировать эти значения через параметр `alpha`.

In [6]:
docs = {item["url"]: (item["title"], item["snippet"]) for item in graph_json}

def print_result(search_result, detail=True):
    TITLE_SETTING = "\033[1;35m{} {}\033[0m"
    
    for url, rank in search_result:
        title, snippet = docs[url]
        print(TITLE_SETTING.format(title, rank))
        if detail:
            print(url)
            print(snippet)
            print()
    

In [30]:
pagerank = nx.pagerank(G, max_iter=200)
print("Max pagerank: %f" % max(pagerank.values()))
print("Min pagerank: %f" % min(pagerank.values()))

Max pagerank: 0.007863
Min pagerank: 0.000016


In [31]:
search_result = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result)

CBS 0.00786279319118392
https://en.wikipedia.org/wiki/CBS
CBS (an initialism of the network's former name, the Columbia Broadcasting System) is an American English language commercial broadcast television network that is a flagship property of CBS Corporation. The company is headquartered at the CBS Building in ...

National Library of the Czech Republic 0.00536512357082438
https://en.wikipedia.org/wiki/National_Library_of_the_Czech_Republic
6,919,075 total items[1]...

Migration Period 0.004846324032569245
https://en.wikipedia.org/wiki/Migration_Period
The Migration Period was a time of widespread migrations of peoples, notably the Germanic tribes and the Huns, within or into Europe in the middle of the first millennium AD. It has also been termed in English by the German loanword Völkerwanderung[2] and...

Federal government of the United States 0.003420131201127228
https://en.wikipedia.org/wiki/Federal_government_of_the_United_States
 ...

National Football League 0.0030185816405407

## 4. Перебор

Параметр альфа - коэффициент затухания.
$$p^{(k+1)} = \alpha P^T p^{(k)}$$

При уменьшении альфы значения весов уменьшаются, но в топ $10$ попадают примерно одни и те же статьи, но мб в разном порядке.

#### alpha=0.95

In [32]:
pagerank = nx.pagerank(G, max_iter=200, alpha=0.95)
print("Max pagerank: %f" % max(pagerank.values()))
print("Min pagerank: %f" % min(pagerank.values()))

Max pagerank: 0.011680
Min pagerank: 0.000005


In [33]:
search_result = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result, detail=False)

CBS 0.011680174241560044
Migration Period 0.01126117634817252
National Library of the Czech Republic 0.008750160474165568
Trade association 0.0065439401734502444
Federal government of the United States 0.006306021547257068
National Institute of Standards and Technology 0.004825462635199798
The Honourable 0.004146829114356125
Yemen 0.0038755613880403993
Public university 0.0036307092044255886
Rocky Mountains 0.0031118403210819083


#### alpha=0.5

In [34]:
pagerank = nx.pagerank(G, max_iter=200, alpha=0.5)
print("Max pagerank: %f" % max(pagerank.values()))
print("Min pagerank: %f" % min(pagerank.values()))

Max pagerank: 0.002591
Min pagerank: 0.000051


In [35]:
search_result = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result, detail=False)

CBS 0.0025914951767385904
National Football League 0.0017498024708572774
National Library of the Czech Republic 0.0014585498875163852
Dutch language 0.0012028404588483213
Biblioteca Nacional de España 0.000943569623814219
Federal government of the United States 0.0009094120038206237
National Library of Australia 0.0008815056584189986
The Honourable 0.000744965195203609
Yemen 0.0007352903525798878
Venice 0.0007161686692459154


#### alpha=0.3

In [36]:
pagerank = nx.pagerank(G, max_iter=200, alpha=0.3)
print("Max pagerank: %f" % max(pagerank.values()))
print("Min pagerank: %f" % min(pagerank.values()))

Max pagerank: 0.001301
Min pagerank: 0.000071


In [37]:
search_result = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result, detail=False)

CBS 0.0013009514398928982
National Football League 0.0010764481714392294
Dutch language 0.0007685523097406355
National Library of the Czech Republic 0.000696281497795263
Biblioteca Nacional de España 0.0006231461154414767
National Library of Australia 0.0005765654672805387
Venice 0.000482374042331735
Federal government of the United States 0.00046856719458300203
Geneva 0.0004047777800338457
The Honourable 0.0004021352654266985


## 5. HITS

__Авторитетный документ__ - документ, на который ссылается много других.

__Хаб-документ__ - документ, содержащий много ссылок на авторитетные документы.

Хоть получается, что идейно авторитетность и pagerank обозначают нечто схожее, но на практике в топе оказались разные документы.

In [26]:
hits = nx.hits(G, max_iter=200)
print("HITS hubs")
print("Max hubs: %f" % max(hits[0].values()))
print("Min hubs: %f" % min(hits[0].values()))
print("HITS authorities")
print("Max authorities: %f" % max(hits[1].values()))
print("Min authorities: %f" % max(hits[1].values()))

HITS hubs
Max hubs: 0.050676
Min hubs: 0.000000
HITS authorities
Max authorities: 0.054090
Min authorities: 0.054090


#### authorities 

In [27]:
search_result = sorted(hits[1].items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result)

Central Japan Railway Company 0.05409019464372503
https://en.wikipedia.org/wiki/Central_Japan_Railway_Company
The Central Japan Railway Company (東海旅客鉄道株式会社, Tōkai Ryokaku Tetsudō Kabushiki-gaisha) is the main railway company operating in the Chūbu (Nagoya) region of central Japan. It is officially abbreviated in English as JR Central and in Japanese as JR Tōkai (...

Biwajima Station 0.05156056180805734
https://en.wikipedia.org/wiki/Biwajima_Station
Biwajima Station (枇杷島駅, Biwajima-eki) is a railway station in Kiyosu, Aichi Prefecture, Japan. The station is a union station served by the Tōkaidō Main Line and the Jōhoku Line. The station is 370.0 rail kilometres from Tokyo Station on the Tōkaidō Main ...

Inazawa Station 0.05019839899978299
https://en.wikipedia.org/wiki/Inazawa_Station
Inazawa Station (稲沢駅, Inazawa-eki) is a railway station in Inazawa, Aichi Prefecture, Japan, on the Tōkaidō Main Line. The station is 377.1 rail kilometers from Tokyo....

Shin-Kambara Station 0.050198398

#### hubs

In [28]:
search_result = sorted(hits[0].items(), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result, detail=False)

Owari-Ichinomiya Station 0.05067649383671277
Atsuta Station 0.0506632750581518
Inazawa Station 0.05051134822709755
Shin-Kambara Station 0.05051134822709755
Kyōwa Station 0.05051134822709755
Otōbashi Station 0.05051134822709755
Kisogawa Station 0.05051134822709755
Ōmi-Nagaoka Station 0.05051134822709755
Samegai Station 0.050063627563415904
Kashiwabara Station 0.04992118770877782


#### avg

In [29]:
search_result = sorted(((k, (hits[0][k] + hits[1][k]) / 2.) for k in nodes), key=lambda x: x[1], reverse=True)[:10]
print_result(search_result, detail=False)

Owari-Ichinomiya Station 0.05043744641824788
Atsuta Station 0.05043083702896739
Inazawa Station 0.050354873613440265
Shin-Kambara Station 0.050354873613440265
Kyōwa Station 0.050354873613440265
Otōbashi Station 0.050354873613440265
Kisogawa Station 0.050354873613440265
Biwajima Station 0.049064500670394175
Nishi-Gifu Station 0.04841888322851266
Yui Station 0.048194037789082295
